In [1]:
from datetime import datetime
import requests
import sys
import numpy as np
import pandas as pd

sys.path.append('..')
import moonlander

from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import moonlander as mld
from moonlander.data.providers import Binance

api_key = 'JUY2Z2R1Lwpgta6toGtB8ofCxpiSBEiqqHJM8rp4GCWzIy2kXFbERuvxB2dDD1Cj'
secret_key = 'hDv51FaiLDHEA7Al2zhuG9YlXoKmgntjvgZDofFKNis1hY27x4WXHoL1RIsf1e8F'

binance = Binance(api_key, secret_key)

In [3]:
# from moonlander.data.providers import CoinDCX

# coindcx_api_key = 'fcb1c800d0ee02c93ea14bf61517afa0194d5c695eec9792'
# coindcx_api_secret = '3071e32a059b72505f7d261d95aacb1ba66ab8e4cf5e89214bf329b454196c4c'

# coindcx = CoinDCX(coindcx_api_key, coindcx_api_secret)

In [4]:
btc = moonlander.core.Asset('DOGE', 'USDT', provider = binance, interval = '4-hour')

All caught up..!


In [5]:
btc.plot_candles()

In [6]:
# btc.start_stream()

In [7]:
# btc.stop_stream()

In [8]:
btc.plot_returns()

In [9]:
btc.plot_returns('hs')

In [10]:
btc.annualized_perf()

Return: 268.822% | Risk: 261.169%


In [11]:
from moonlander.strategy import TradingStrategyBase

# Build mean reversion strategy
from talib.abstract import *

def bands(df, timeperiod=26, nbdevup=2.6, nbdevdn=2.6, matype=0):
    cols = ['high', 'low', 'open', 'close', 'volume']
    HLOCV = {key: df[key].values for key in df if key in cols}
    u, m, l = BBANDS(HLOCV, timeperiod=timeperiod, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=matype)
    df['upper']  = u
    df['middle'] = m
    df['lower']  = l
    return df

def touches(df):
    df['touch_upper'] = df.high >= df.upper
    df['touch_lower'] = df.low  <= df.lower
    df['crossing_dn'] = (df.close < df.middle) & (df.open > df.middle)
    df['crossing_up'] = (df.close > df.middle) & (df.open < df.middle)
    return df

class MeanReversion(TradingStrategyBase):
    def logic(self, account, data):
        try:
            data = bands(data)
            data = touches(data)
            today = data.iloc[-1]
            
            # Buying
            if today.touch_lower | today.crossing_dn:
                risk          = 1
                entry_price   = today.lower
                entry_capital = account.buying_power*risk
                if entry_capital > 0:
                    account.buy(entry_capital=entry_capital, 
                                           entry_price=entry_price,
                                          commission = 0.01)
            elif today.crossing_dn:
                risk          = 1
                entry_price   = today.close
                entry_capital = account.buying_power*risk
                if entry_capital > 0:
                    account.buy(entry_capital=entry_capital, 
                                           entry_price=entry_price,
                                          commission = 0.01)
                    
            # Selling
            elif today.touch_upper:
                exit_price = today.upper
                account.sell(percent=1.0, current_price=exit_price, commission = 0.01)
                        
            elif today.crossing_up:
                exit_price = today.close
                account.sell(percent=1.0, current_price=exit_price, commission = 0.01)
                    
        except Exception as e:
            print(e)
            pass # Handles lookback errors in beginning of dataset

In [12]:
btc.data = btc.data.loc['2020-01-01 00:00:00':]

In [13]:
strat = MeanReversion(btc)

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
strat.backtest()

Error: Entry price cannot be negative.

-------------- Results ----------------

Buy and Hold : 21523.96%
Net Profit   : 215239.57
Strategy     : 27336.85%
Net Profit   : 273368.53
Buys        : 142
Sells        : 142
--------------------
Total Trades : 284

---------------------------------------
